<a href="https://colab.research.google.com/github/keatrean/projects/blob/master/get_swearwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
data = pd.read_csv('train.csv')

In [ ]:
data = data.reset_index()
print(data.head())

   index  ID                                               text label
0      0   0  брюки отличные, качественные, но к сожалению к...   NaN
1      1   1  отличный аппарат, в комплекте кабель и работет...   NaN
2      2   2                            супер 👍 спасибо большое   NaN
3      3   3  получил быстро данные наушники! к наушникам пр...   NaN
4      4   4                 всё дошло в целости и сохранности)   NaN


In [ ]:
import re
import pandas as pd

# Регулярное выражение для поиска мата
bad_words_pattern = re.compile(r"""(?iux)(?<![а-яё])(?:
(?:
(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])-?)?(?:
  [её](?:б(?!о[рй]|рач)|п[уа](?:ц|тс))|
  и[пб][ае][тцд][ьъ]
).*?|

(?:(?:н[иеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еи])?|а[пб]ч|в[ъы]?|пр[еи])-?)?ху(?:[яйиеёю]|л+и(?!ган)).*?|

бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|

\S*?(?:
  п(?:
    [иеё]зд|
    ид[аое]?р|
    ед(?:р(?!о)|[аое]р|ик)|
    охую
  )|
  бля(?:[дбц]|тс)|
  [ое]ху[яйиеё]|
  хуйн
).*?|

(?:о[тб]?|про|на|вы)?м(?:
  анд(?:[ауеыи](?:л(?:и[сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|ища)|
  уд(?:[яаиое].+?|е?н(?:[ьюия]|ей))|
  [ао]л[ао]ф[ьъ](?:[яиюе]|[еёо]й)
)|

елд[ауые].*?|
ля[тд]ь|
(?:[нз]а|по)х
)(?![а-яё])""", re.IGNORECASE)


# Функция обработки текста
def process_text(text):
    found_words = bad_words_pattern.findall(text)
    text = bad_words_pattern.sub("", text)  # Удаляем найденные слова
    label = ", ".join(found_words) if found_words else None
    return text.strip(), label


# Применяем обработку к каждому комментарию
data[["text", "label"]] = data["text"].apply(lambda x: process_text(str(x))).apply(pd.Series)

# Сохраняем результаты
data.to_csv("filtered_comments.csv", index=False)

In [ ]:
import re
def preprocess_text(text):
    text = text.lower()
    text = remove_emojis(text)
    return text

def remove_emojis(text):
    regrex_pattern = re.compile(pattern = "["
                                u"\U00000000-\U00000009"
                                u"\U0000000B-\U0000001F"
                                u"\U00000080-\U00000400"
                                u"\U00000402-\U0000040F"
                                u"\U00000450-\U00000450"
                                u"\U00000452-\U0010FFFF"
                                "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)



data['text'] = data['text'].apply(preprocess_text)

In [ ]:
data.head()

,index,ID,text,label
0,0,0,"брюки отличные, качественные, но к сожалению к...",NaN
1,1,1,"отличный аппарат, в комплекте кабель и работет...",NaN
2,2,2,супер спасибо большое,NaN
3,3,3,получил быстро данные наушники! к наушникам пр...,NaN
4,4,4,всё дошло в целости и сохранности),NaN


In [ ]:
def clean_token(token):
    return token.replace('?', '').replace(',', '').replace('!', '').replace('+', '').replace('(', '').replace(')', '')

def create_ner_tags(data):
    sentences = []
    labels = []

    for _, row in data.iterrows():
        text = row['text']
        bad_words = row['label'].split(',') if pd.notna(row['label']) else []
        # Разбиваем текст на токены и очищаем каждый токен
        tokens = [clean_token(token) for token in text.split()]
        tokens = [token for token in tokens if token.strip()]
        token_labels = ["O"] * len(tokens)

        for word in bad_words:
            for i, token in enumerate(tokens):
                if token.startswith(word):
                    token_labels[i] = "B-SWEARWORD" if token_labels[i] == "O" else "I-SWEARWORD"

        sentences.append(tokens)
        labels.append(token_labels)

    return sentences, labels

sentences, labels = create_ner_tags(data)


In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({"ID": data['ID'], "sentences": sentences, "labels": labels})

In [ ]:
dataset

Dataset({
    features: ['ID', 'sentences', 'labels'],
    num_rows: 244739
})

In [ ]:
print(dataset[0])

{'ID': 0, 'sentences': ['брюки', 'отличные', 'качественные', 'но', 'к', 'сожалению', 'коротко', 'ваты', 'поэтому', 'отказ', 'обидно', 'продавца', 'рекомендую', 'всем'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [ ]:
from transformers import BertTokenizerFast

model_name = "DeepPavlov/rubert-base-cased-conversational"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

def tokenize_and_align_labels(sentences, labels, label_list):
    tokenized_inputs = tokenizer(sentences, truncation=True, is_split_into_words=True, padding=True, max_length=128)
    labels_aligned = []

    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = [-100 if word_id is None else label_list.index(label[word_id]) for word_id in word_ids]
        labels_aligned.append(aligned_labels)

    tokenized_inputs["labels"] = labels_aligned
    return tokenized_inputs


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [ ]:
label_list = ["O", "B-SWEARWORD", "I-SWEARWORD"]
tokenized_data = dataset.map(lambda x: tokenize_and_align_labels(x["sentences"], x["labels"], label_list), batched=True)

Map:   0%|          | 0/244739 [00:00<?, ? examples/s]

In [ ]:
sentence_idx = 0
tokens = tokenizer.convert_ids_to_tokens(tokenized_data['input_ids'][sentence_idx])
labels = tokenized_data['labels'][sentence_idx]

print(f"Sentence: {' '.join(sentences[sentence_idx])}")
print(f"TOKENS: {tokens}")
print(f"LABELS: {labels}")

Sentence: брюки отличные качественные но к сожалению коротко ваты поэтому отказ обидно продавца рекомендую всем
TOKENS: ['[CLS]', 'брюки', 'отличные', 'качественные', 'но', 'к', 'сожалению', 'коротко', 'ваты', 'поэтому', 'отказ', 'обидно', 'продавца', 'рекомендую', 'всем', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [ ]:
sentence_idx = 74
tokens = tokenizer.convert_ids_to_tokens(tokenized_data['input_ids'][sentence_idx])
labels = tokenized_data['labels'][sentence_idx]
print(f"Sentence: {' '.join(sentences[sentence_idx])}")
print(f"TOKENS: {tokens}")
print(f"LABELS: {labels}")

Sentence: полное г...но клей весь высохший. зря не поверила отзывам других покупателей.
TOKENS: ['[CLS]', 'полное', 'г', '.', '.', '.', 'но', 'клей', 'весь', 'высох', '##ший', '.', 'зря', 'не', 'поверила', 'отзывам', 'других', 'покупателей', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [ ]:
tokenized_data

Dataset({
    features: ['ID', 'sentences', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 244739
})

In [ ]:
print(tokenized_data[1])

{'ID': 1, 'sentences': ['отличный', 'аппарат', 'в', 'комплекте', 'кабель', 'и', 'работетт', 'от', 'сети', 'а', 'значит', 'можно', 'сразу', 'приступать', 'к', 'измерению', 'давления.'], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 5682, 9844, 340, 5007, 4925, 322, 3996, 8301, 847, 5925, 301, 1993, 1088, 18

In [ ]:
from datasets import DatasetDict

train_test_split = tokenized_data.train_test_split(test_size=0.2, seed=42)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['ID', 'sentences', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 195791
    })
    test: Dataset({
        features: ['ID', 'sentences', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 48948
    })
})


In [ ]:
test_dataset = train_test_split['test']

In [ ]:
test_dataset

Dataset({
    features: ['ID', 'sentences', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 48948
})

In [ ]:
print(dataset_dict['train'][0])

{'ID': 41886, 'sentences': ['отлично', 'работает', 'в', 'составе', 'смеси', 'с', 'простадонтом', 'и', 'поликавином.', 'хорошо', 'при', 'недержании', 'мочи', 'как', 'у', 'мужчин', 'так', 'и', 'у', 'женщин.', 'рекомендую.'], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 4065, 2587, 340, 5892, 27666, 336, 21707, 6051, 1056, 322, 1355, 86695,

In [ ]:
from transformers import BertForTokenClassification
# Загрузка модели
model = BertForTokenClassification.from_pretrained("DeepPavlov/rubert-base-cased-conversational", num_labels=len(label_list))

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(dataset_dict['train'][23]["input_ids"]), dataset_dict['train'][23]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
юбка        0
супер       0
по          0
фигуре      0
села        0
идеально    0
.           0
рекомендую  0
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=2e410ba4b25e0b5d9388993a5e1bfabcb7a35c8c7648a4a93f6a41be7b46c1b8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import evaluate
import numpy as np


metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }



#Загрузка моей модели

# not available  (NDA)

# Обучение:

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    output_dir="./bert_ner_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    fp16=True,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-35-545aed597b95>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.006200,0.007577,0.935252,0.909690,0.922294,0.997815
2,0.028400,0.007286,0.939621,0.918581,0.928982,0.997990
3,0.002200,0.008074,0.930108,0.932329,0.931217,0.998045


TrainOutput(global_step=18357, training_loss=0.007310315687214441, metrics={'train_runtime': 4772.4004, 'train_samples_per_second': 123.077, 'train_steps_per_second': 3.846, 'total_flos': 3.837001187136845e+16, 'train_loss': 0.007310315687214441, 'epoch': 3.0})

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `keatrean` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when 

In [ ]:
trainer.save_model("./swearwords_detection_model")

#Загрузка данных submission Kaggle

In [ ]:
import pandas as pd
submission_data = pd.read_csv('test.csv',index_col=0, skip_blank_lines=False)

In [ ]:
submission_data.head()

,text
ID,
0,"хороший, подошкл"
1,"совсем тонюсенький саженец, не досмотрела в оп..."
2,"когтеточка хорошая, но вот ткань на основании ..."
3,"много затяжек, не порадовала покупка"
4,рекомендую 💣


In [ ]:
with open('test.csv') as f:
    lines = f.readlines()
print(f"Исходное количество строк в файле: {len(lines)}")
print(f"Количество строк в DataFrame: {len(submission_data)}")


Исходное количество строк в файле: 68927
Количество строк в DataFrame: 66949


In [ ]:
submission_data = submission_data.reset_index()

print(submission_data.head())

   ID                                               text
0   0                                   хороший, подошкл
1   1  совсем тонюсенький саженец, не досмотрела в оп...
2   2  когтеточка хорошая, но вот ткань на основании ...
3   3               много затяжек, не порадовала покупка
4   4                                       рекомендую 💣


In [ ]:
import re
import pandas as pd

# Регулярное выражение для поиска мата
bad_words_pattern = re.compile(r"""(?iux)(?<![а-яё])(?:
(?:
(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])-?)?(?:
  [её](?:б(?!о[рй]|рач)|п[уа](?:ц|тс))|
  и[пб][ае][тцд][ьъ]
).*?|

(?:(?:н[иеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еи])?|а[пб]ч|в[ъы]?|пр[еи])-?)?ху(?:[яйиеёю]|л+и(?!ган)).*?|

бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|

\S*?(?:
  п(?:
    [иеё]зд|
    ид[аое]?р|
    ед(?:р(?!о)|[аое]р|ик)|
    охую
  )|
  бля(?:[дбц]|тс)|
  [ое]ху[яйиеё]|
  хуйн
).*?|

(?:о[тб]?|про|на|вы)?м(?:
  анд(?:[ауеыи](?:л(?:и[сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|ища)|
  уд(?:[яаиое].+?|е?н(?:[ьюия]|ей))|
  [ао]л[ао]ф[ьъ](?:[яиюе]|[еёо]й)
)|

елд[ауые].*?|
ля[тд]ь|
(?:[нз]а|по)х|

# --- Добавленные фильтры ---
(?:г[оа]вн(?:[оаыеи]|юк|ищ|ар|ист|юч|ёш|ул).*?)|  # Фильтрация слов с "говн"
(?:жоп(?:[ауыеи]|аст|ищ|ник|арь|ист|к).*?)|        # Фильтрация слов с "жоп"
(?:дерьм(?:о|ец|овый|ища|ище|ецкий|оед|яный).*?)|  # Фильтрация слов с "дерьм"
(?:залуп(?:а|у|ы|е|ой|ищ|н).*?)|                   # Фильтрация слов с "залуп"
(?:сук(?:а|о|и|ище|ин|арь|он).*?)|                 # Фильтрация слов с "сук"
(?:пидар(?:ас|юга|южка|ок|к.*?|ов).*?)|            # Фильтрация слов с "пидар"
(?:збс)|                                           # Фильтрация "збс" (жаргонное)
(?:хз(?![а-яё]))|                                  # Фильтрация "хз" (но без слов, содержащих его)
(?:сра(?:ка|т).*?)                                 # Фильтрация слов "срака" или "срат"
)(?![а-яё])""", re.IGNORECASE)




# Функция обработки текста
def process_text(text):
    found_words = bad_words_pattern.findall(text)
    text = bad_words_pattern.sub("", text)  # Удаляем найденные слова
    label = ", ".join(found_words) if found_words else None
    return text.strip(), label


# Применяем обработку к каждому комментарию
submission_data[["text", "label"]] = submission_data["text"].apply(lambda x: process_text(str(x))).apply(pd.Series)

# Сохраняем результаты
submission_data.to_csv("filtered_testdata.csv", index=False)

In [ ]:
import re

def preprocess_text(text):
    text = text.lower()
    text = remove_emojis(text)
    tokens = text.split()
    tokens = remove_punctuation(tokens)
    return tokens

def remove_punctuation(tokens):
    processed_tokens = []
    for token in tokens:
        # Удаляем пунктуацию из каждого токена
        token = token.replace(',', ' ').replace('!', ' ').replace('?', ' ')
        token = token.replace('(', ' ').replace(')', ' ')
        token = token.replace('+', ' ')
        processed_tokens.append(' '.join(token.split()))  # Добавляем обработанный токен в список
    return processed_tokens

def remove_emojis(text):
    regrex_pattern = re.compile(pattern = "["
                                u"\U00000000-\U00000009"
                                u"\U0000000B-\U0000001F"
                                u"\U00000080-\U00000400"
                                u"\U00000402-\U0000040F"
                                u"\U00000450-\U00000450"
                                u"\U00000452-\U0010FFFF"
                                "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

submission_data['sentences'] = submission_data['text'].apply(preprocess_text)


In [ ]:
from datasets import Dataset

submission_dataset = Dataset.from_dict({
    'ID': submission_data['ID'].tolist(),
    'sentences': submission_data['sentences'].tolist()
})

print(submission_dataset)

Dataset({
    features: ['ID', 'sentences'],
    num_rows: 66949
})


In [ ]:
print(submission_dataset[0])

{'ID': 0, 'sentences': ['хороший', 'подошкл']}


In [ ]:
submission_dataset

Dataset({
    features: ['sentences'],
    num_rows: 66949
})

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('')

def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['sentences'], truncation=True, is_split_into_words=True, padding=True, max_length=128)
    word_ids = [tokenized_inputs.word_ids(i) for i in range(len(examples['sentences']))]
    tokenized_inputs['word_ids'] = word_ids
    return tokenized_inputs

tokenized_submission_dataset = submission_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/66949 [00:00<?, ? examples/s]

In [ ]:
tokenized_submission_dataset

Dataset({
    features: ['ID', 'sentences', 'input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 66949
})

In [ ]:
import numpy as np
# Получение предсказаний
predictions, labels, _ = trainer.predict(tokenized_submission_dataset)

In [ ]:
predictions = np.argmax(predictions, axis=2)

In [ ]:
import numpy as np
# Получение предсказаний
test_predictions, test_labels, _ = trainer.predict(test_dataset)

NameError: name 'test_dataset' is not defined

In [ ]:
test_predictions = np.argmax(test_predictions, axis=2)

In [ ]:
import numpy as np

In [ ]:
np.save("predictions.npy", predictions)

In [ ]:
test_predictions = np.load('test_predictions.npy')

In [ ]:
def extract_named_entities(tokens, labels):

    entities = []
    current_entity = []

    special_tokens = {'[CLS]', '[SEP]', '[PAD]'}  # Набор специальных токенов

    for token, label in zip(tokens, labels):
        if token in special_tokens:  # Пропустить специальные токены
            continue

        if label > 0:  # Если метка указывает на часть сущности
            if token.startswith("##"):  # Если токен начинается с ##, объединяем его с предыдущим
                if current_entity:
                    current_entity[-1] += token[2:]  # Удаляем '##' и добавляем к предыдущему токену
                else:
                    current_entity.append(token[2:])  # Если первая сущность, начать новую
            else:
                current_entity.append(token)
        elif current_entity:  # Завершение текущей сущности
            entities.append("".join(current_entity))
            current_entity = []

    # Добавить последнюю сущность, если она есть
    if current_entity:
        entities.append("".join(current_entity))

    return entities




In [ ]:
def save_predictions(predictions, dataset, tokenizer, output_file="submission.csv"):
    results = []
    for i, item in enumerate(dataset):  # Итерация по текстам
        tokens = tokenizer.convert_ids_to_tokens(item['input_ids'])
        preds = predictions[i]

        # Извлечение матерных слов из BERT
        bert_bad_words = extract_named_entities(tokens, preds)

        # Объединяем с уже найденными словами из регулярок
        existing_label = submission_data.loc[submission_data["ID"] == item["ID"], "label"].values[0]
        all_bad_words = set(bert_bad_words)
        if existing_label:
            all_bad_words.update(existing_label.split(", "))

        # Формируем строку для DataFrame
        result_row = {
            "ID": item['ID'],  # ID текста
            "label": ", ".join(sorted(all_bad_words)) if all_bad_words else ""
        }
        results.append(result_row)

    df_results = pd.DataFrame(results)
    df_results.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Результаты сохранены в {output_file}")

In [ ]:
save_predictions(predictions, tokenized_submission_dataset, tokenizer, "submission.csv")


Результаты сохранены в submission.csv


In [ ]:
import pandas as pd

def save_test_predictions(predictions, dataset, tokenizer, output_file="test_submission.csv"):

    results = []
    for i, item in enumerate(dataset):  # Итерация по текстам
        tokens = tokenizer.convert_ids_to_tokens(item['input_ids'])
        preds = predictions[i]

        # Извлечение матерных слов из BERT
        bert_bad_words = extract_named_entities(tokens, preds)

        # Объединяем с уже найденными словами из регулярок
        existing_label = data.loc[data["ID"] == item["ID"], "label"].values[0]
        all_bad_words = set(bert_bad_words)
        if existing_label:
            all_bad_words.update(existing_label.split(", "))

        # Формируем строку для DataFrame
        result_row = {
            "ID": item['ID'],  # ID текста
            "label": ",".join(sorted(all_bad_words)) if all_bad_words else "",
            'true_label': data.loc[data['ID'] == item['ID'], 'label'].values[0],
            'text': data.loc[data['ID'] == item['ID'], 'text'].values[0]
        }
        results.append(result_row)

    df = pd.DataFrame(results)

    df.to_csv(output_file, index=False, encoding="utf-8")

    print(f"Результаты сохранены в {output_file}")

In [ ]:
save_test_predictions(test_predictions, test_dataset, tokenizer, "test_submission.csv")


Результаты сохранены в test_submission.csv
